### Import libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tensorflow.keras import layers
from sklearn.model_selection import KFold

In [2]:
from keras import metrics

In [3]:
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error

In [4]:
import time

### Import dataset

In [9]:
whole_data = pd.read_csv("Without_Outliers2.csv") 

In [10]:
whole_data.head()

,Unnamed: 0,Year,Location,Wind_Direction,Wind_Speed,temperature_2m,relativehumidity_2m,dewpoint_2m,surface_pressure,et0_fao_evapotranspiration,...,soil_moisture_28_to_100cm,direct_radiation,diffuse_radiation,rain,weathercode,windgusts_10m,cloudcover,Production,Area,Yield
0,0,1990,AGUSAN DEL NORTE,6,4.1,25.15,78.360000,20.88,975.30,0.16,...,0.28,140.02,70.89,1701.0,1,9.4,100.0,226.68,200.0,1.133400
1,1,1991,AGUSAN DEL NORTE,8,3.1,25.06,77.990000,20.73,975.55,0.16,...,0.27,138.79,73.80,1634.2,1,11.5,100.0,231.68,204.0,1.135686
2,2,1992,AGUSAN DEL NORTE,8,3.1,25.39,77.139973,20.11,975.90,0.17,...,0.21,144.62,73.53,1433.2,1,7.9,100.0,219.08,199.0,1.100905
3,3,1993,AGUSAN DEL NORTE,8,3.6,24.92,79.600000,20.94,975.66,0.15,...,0.28,136.18,72.02,2017.8,1,8.3,100.0,231.47,203.0,1.140246
4,4,1994,AGUSAN DEL NORTE,6,3.6,24.96,79.450000,20.96,975.32,0.15,...,0.29,137.68,71.24,1722.0,1,8.6,100.0,238.09,208.0,1.144663


In [11]:
whole_data.columns

Index(['Unnamed: 0', 'Year', 'Location', 'Wind_Direction', 'Wind_Speed',
       'temperature_2m', 'relativehumidity_2m', 'dewpoint_2m',
       'surface_pressure', 'et0_fao_evapotranspiration',
       'vapor_pressure_deficit', 'soil_temperature_28_to_100cm',
       'soil_moisture_28_to_100cm', 'direct_radiation', 'diffuse_radiation',
       'rain', 'weathercode', 'windgusts_10m', 'cloudcover', 'Production',
       'Area', 'Yield'],
      dtype='object')

### Define features and target

In [12]:
#define target and predictors 

X = whole_data[['weathercode', 'temperature_2m',
       'relativehumidity_2m', 'dewpoint_2m', 'rain',
       'surface_pressure', 'cloudcover',
       'et0_fao_evapotranspiration', 'vapor_pressure_deficit',
       'Wind_Direction', 'Wind_Speed', 'windgusts_10m',
       'soil_temperature_28_to_100cm',
       'soil_moisture_28_to_100cm', 'direct_radiation',
       'diffuse_radiation']]

In [13]:
X.columns

Index(['weathercode', 'temperature_2m', 'relativehumidity_2m', 'dewpoint_2m',
       'rain', 'surface_pressure', 'cloudcover', 'et0_fao_evapotranspiration',
       'vapor_pressure_deficit', 'Wind_Direction', 'Wind_Speed',
       'windgusts_10m', 'soil_temperature_28_to_100cm',
       'soil_moisture_28_to_100cm', 'direct_radiation', 'diffuse_radiation'],
      dtype='object')

In [8]:
y = whole_data['Yield (mt/ha)']

### MLR Model

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
from sklearn.linear_model import LinearRegression
def MLR_Model(X_train,y_train):
    mlr = LinearRegression()
    mlr.fit(X_train,y_train)
    return mlr

#### Create ANN Model

In [11]:
from tensorflow.keras import activations

In [12]:
def ANN_Model (initial_weights,bias,neurons = 1,learning_rate=0.01, input_shape=16, activation=activations.relu): 
    model = Sequential()
    model.add(Input(shape = X_train.shape[1])) # number of input features 
    model.add(Dense(neurons, activation = activation, name = 'hidden')) # one neuron in hidden layer, relu activation
    model.add(Dense(1,activation = 'linear')) # one neuron in output layer, linear activation


    #initialize weights and bias in hidden layer
    hidden_layer = model.get_layer('hidden')
    #layer.set_weights([weights_array, bias_array])
    hidden_layer.set_weights([np.array([initial_weights]).reshape((16, 1)), np.array([bias])])


    #compile 
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics = [metrics.mean_absolute_error,tf.keras.metrics.RootMeanSquaredError()])
    return model



### Hybrid Model Cross Validation 

In [13]:
start_time = time.time()

#initializing KFold object
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
mae_scores = []
r2_scores = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlr_model = MLR_Model(X_train,y_train) #perform MLR
    ann_model = ANN_Model(mlr_model.coef_,mlr_model.intercept_) #Intercept and coeff of MLR will be initailzied as ANN weights
    ann_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0) #fit ann model to data
    print(X_train)
    print(ann_model.get_weights())
    y_pred = ann_model.predict(X_test) #use ann model to predict test set
    print(f"MAE: { mean_absolute_error(y_test, y_pred)}")  #measure mean absolute value
    print(f"R2: {r2_score(y_test,y_pred)}")#measure R-squared value
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print(f"Intercept: {mlr_model.intercept_}")
    print(f"Coefficients: {list(zip(mlr_model.coef_))}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


[[-0.41033963  0.08986576 -2.52535882 ... -1.83998537 -0.03583865
  -0.410207  ]
 [-0.41033963  0.04069794 -2.67391028 ... -2.00010087 -0.12189875
   0.52507965]
 [-0.41033963  0.22097996 -3.01518787 ... -2.96079389  0.28601212
   0.43830047]
 ...
 [-0.41033963  0.54330236 -0.14853549 ...  0.56174717  0.3524813
   0.51865156]
 [-0.41033963  0.60339636  0.09637367 ...  0.72186267 -0.37798001
   0.63757117]
 [-0.41033963  0.42311434  0.89132471 ...  1.36232468 -1.4072028
   1.96497111]]
[array([[ 0.23050131],
       [-0.5589194 ],
       [-0.99432224],
       [ 0.37249658],
       [-0.85521823],
       [-0.53018796],
       [-0.35795632],
       [ 0.09096181],
       [-0.8764771 ],
       [-0.23161574],
       [ 0.2166952 ],
       [ 0.2510235 ],
       [ 0.29725218],
       [-0.11424324],
       [-0.24520785],
       [ 0.54139066]], dtype=float32), array([2.3570635], dtype=float32), array([[0.25202137]], dtype=float32), array([0.11905321], dtype=float32)]
3/3 [==========================

### saving the model

In [14]:
import pickle

In [15]:
filename = 'trained_hybrid_model.sav'

In [16]:
pickle.dump(ann_model, open(filename, 'wb'))

In [17]:
loaded_model = pickle.load(open('trained_hybrid_model.sav', 'rb'))

### Attempt 1

This is the first entry in the dataset. 

In [127]:
input_data = (1.00, 25.15,28.36,20.88,1701.00,975.30,100.00,0.16,0.75,6.00,4.10,9.40,26.01,0.28,140.02, 70.89)

In [128]:
input_data_as_numpy_array = np.asarray(input_data)

In [129]:
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

In [131]:
prediction = loaded_model.predict(input_data_reshaped)
print(f"Predicted output: {prediction}")

1/1 [==============================] - 0s 64ms/step
Predicted output: [[0.1334619]]


In [122]:
print(f"Actual output: {y.iloc[0]}")

Actual output: 1.1334


### Attempt 2 

The input data is based from the dataset but the values are rounded to the nearest whole number

In [116]:
input_data_2 = (1, 25,28,21,1701,975,100,0.16,0.75,6.00,4,9,26,0.28,140, 71)

In [117]:
input_data_as_numpy_array_2 = np.asarray(input_data_2)

In [118]:
input_data_reshaped_2 = input_data_as_numpy_array_2.reshape(1,-1)

In [120]:
prediction = loaded_model.predict(input_data_reshaped_2)
print(f"Predicted output: {prediction}")

1/1 [==============================] - 0s 50ms/step
Predicted output: [[0.1334619]]


In [121]:
print(f"Actual output: {y.iloc[0]}")

Actual output: 1.1334


# Dont mind this section (wala pa nahuman)

### Leave one out

This is working pero dugay siya because k = number of data points. 
* R2 is not showing (Error: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.)

In [65]:
import time
from sklearn.model_selection import LeaveOneOut

In [68]:
start_time = time.time()

#initializing KFold object
loo = LeaveOneOut()
mae_scores = []
r2_scores = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlr_model = MLR_Model(X_train,y_train) #perform MLR
    ann_model = ANN_Model(mlr_model.coef_,mlr_model.intercept_) #Intercept and coeff of MLR will be initailzied as ANN weights
    ann_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0) #fit ann model to data
    y_pred = ann_model.predict(X_test) #use ann model to predict test set
    print(f"MAE: { mean_absolute_error(y_test, y_pred)}")  #measure mean absolute value
    print(f"R2: {r2_score(y_test,y_pred)}")#measure R-squared value
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print(f"Intercept: {mlr_model.intercept_}")
    print(f"Coefficients: {list(zip(mlr_model.coef_))}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

1/1 [==============================] - 0s 89ms/step
MAE: 0.1897932657241821
R2: nan
RMSE: 0.1897932657241821
Intercept: 0.7870085949035477
Coefficients: [(0.05132091500095425,), (-0.2669382047324648,), (-0.5922470067350075,), (0.26248723596115053,), (-0.21303378314670382,), (-0.16216237421957952,), (-0.05836887977094923,), (0.11550343385336188,), (-0.5872222794647664,), (-0.03968618408762144,), (0.03321198908115678,), (0.044475828372272326,), (0.20906991531596894,), (-0.007694453003741271,), (-0.06917070404647602,), (0.1610333627880207,)]


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 104ms/step
MAE: 1.318401616448784
R2: nan
RMSE: 1.318401616448784
Intercept: 0.787005538386169
Coefficients: [(0.051170082446475924,), (-0.25671789576490134,), (-0.592782069124299,), (0.2552215744379957,), (-0.2130908595666721,), (-0.1622081360112782,), (-0.058302166811070755,), (0.11620431138894044,), (-0.590291187878203,), (-0.039519107708693504,), (0.03326814787228202,), (0.04446726457735031,), (0.2071771166546249,), (-0.00816981883270756,), (-0.06974064016790059,), (0.16139771578721532,)]


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 88ms/step
MAE: 0.36607593165325936
R2: nan
RMSE: 0.36607593165325936
Intercept: 0.7870520380546138
Coefficients: [(0.051586643658591516,), (-0.3348994612032508,), (-0.6027896373998187,), (0.32527970863649414,), (-0.21360603569079684,), (-0.16451642474973843,), (-0.05828022438266153,), (0.11533928430588489,), (-0.576507460793761,), (-0.03991720075036912,), (0.03240476565686133,), (0.04456180617946013,), (0.2092894107099519,), (-0.006957808547095334,), (-0.06956817384942085,), (0.1604094271371959,)]


...

1/1 [==============================] - 0s 91ms/step
MAE: 0.47428161378741457
R2: nan
RMSE: 0.47428161378741457
Intercept: 0.7879619635225301
Coefficients: [(0.05119473577257906,), (-0.2776041919609707,), (-0.5975944826978705,), (0.2745716611214835,), (-0.21210763487459328,), (-0.15958984951652885,), (-0.05778827373632408,), (0.12194147117889824,), (-0.5898353188683491,), (-0.04037627703696145,), (0.031123135955703554,), (0.045110359722762526,), (0.20352639873736272,), (-0.010143950200196167,), (-0.07069272493011611,), (0.16260706321280177,)]


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 176ms/step
MAE: 0.297938186291626
R2: nan
RMSE: 0.297938186291626
Intercept: 0.7879660515586263
Coefficients: [(0.05137004161058178,), (-0.2907849546668445,), (-0.5910659939290754,), (0.282688727077964,), (-0.21289106113457776,), (-0.16145814978538897,), (-0.05824688842825581,), (0.11367463241711073,), (-0.5756875853036073,), (-0.03883005749548382,), (0.033296542971537856,), (0.04465373506914292,), (0.20636661871309048,), (-0.009235393219510958,), (-0.06568953177140394,), (0.1629512109219434,)]


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


KeyboardInterrupt: 

### Tuning

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
pip install scikeras[tensorflow]

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.12.1-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.1-cp311-cp311-win_amd64.whl (272.9 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB 991.0 kB/s eta 0:00:02
     -                                        0.1/1.7 MB 919.0 kB/s eta 0:00:02
     -                                        0.1/1.7 MB 919.0 kB/s eta 0:00:02
     ---                                      0.1/1.7 MB 711.9 kB/s eta 0:00:03
     -----                                    0.2/1.7 MB 919.0 kB/s eta 0:00:02
     ------                                   0.3/1.7 MB 874.6 kB/s eta 0:00:02
     --------                                 0.4/1.7 MB 1.1 MB/s eta 0:00:02
     --------                                 0.4/1.7 MB 998.3 kB/s eta 0:00:02
     --------    

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python311\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
from scikeras.wrappers import KerasRegressor

In [ ]:
import numpy as np

In [ ]:
from sklearn.pipeline import Pipeline

In [59]:
start_time = time.time()

#initializing KFold object
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)

#param_grid
#neuron = (1,3,5,7,9)
#lr=  (0.001, 0.01, 0.1, 0.2, 0.3)
#activation_func= [activations.relu,activations.tanh,activations.linear]
#param_grid= dict(neurons= neurons, learning_rate=lr,activation = activation_func)
#units = list(np.arange(1,5))

param_grid = {'model__units': list(np.arange(1,5)),
              'optimizer__lr' : list(np.arange(0.01,0.05)),
              'activation' :['relu','tanh','linear']}

In [67]:
import os

In [68]:
path_checkpoint = "training_1/cp.ckpt"
directory_checkpoint = os.path.dirname(path_checkpoint)

callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=1)

In [70]:

for train_index, test_index in kfold.split(X):
    # split data

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # configure the cross-validation procedure

    #define model 
    mlr_model = MLR_Model(X_train,y_train) #perform MLR model 
    #vector = np.vectorize(np.int_)
    #coef = vector(np.array(mlr_model.coef_))
    ann_model = ANN_Model(list(mlr_model.coef_),mlr_model.intercept_) #Intercept and coeff of MLR will be initailzied as ANN weights
    model = KerasRegressor(model=ann_model,
                loss = 'mean_absolute_error',
                optimizer= 'adam',
                optimizer__lr=0.01,
                model__units=(1,),
                activation= 'relu',
                epochs=500, 
                batch_size=32, 
                verbose=False)


    
    #define search 
    
    gs=GridSearchCV(estimator = model, param_grid = param_grid,cv=cv_inner,n_jobs = -1,scoring  = 'r2',verbose =True) 
    
    path_checkpoint = "training_1/cp.ckpt"
    directory_checkpoint = os.path.dirname(path_checkpoint)

    callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=1)
    #execute search
    gs.fit(X_train, y_train) #fit ann model to data
    print(gs.best_score_, gs.best_params_)

    
    # get the best performing model fit on the whole training set
    '''
    print(f"R2: {r2_score(y_test,y_pred)}")#measure R-squared value
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    '''


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpnrwg0asy\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpnrwg0asy\assets


Fitting 3 folds for each of 12 candidates, totalling 36 fits
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpyeo65pll\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpyeo65pll\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpva3y3r9f\assets


...

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpoyilqjy0\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpoyilqjy0\assets


KeyboardInterrupt: 

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp1u3obzt5\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp1u3obzt5\assets


In [ ]:
import sklearn

In [ ]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
pip install keras==2.12.0

In [19]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install --user -U scikit-learn==0.21.2

### Traditional ANN
This is working (no problems encountered sana oil)

In [71]:
def Trad_ANN_Model (units = 1,learning_rate = 0.01,activation = 'relu'): 
    model = Sequential()
    model.add(Input(shape = (X_train.shape[1],))) # number of input features 
    model.add(Dense(units=units, activation = activation)) # one neuron in hidden layer, relu activation
    model.add(Dense(1,activation = 'linear')) # one neuron in output layer, linear activation

    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics = [metrics.mean_absolute_error,tf.keras.metrics.RootMeanSquaredError()])
    return model

In [74]:
start_time = time.time()

#initializing KFold object
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
mae_scores = []
r2_scores = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlr_model = MLR_Model(X_train,y_train) #perform MLR
    ann_model = Trad_ANN_Model() #Intercept and coeff of MLR will be initailzied as ANN weights
    ann_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0) #fit ann model to data
    y_pred = ann_model.predict(X_test) #use ann model to predict test set
    print(f"MAE: { mean_absolute_error(y_test, y_pred)}")  #measure mean absolute value
    print(f"R2: {r2_score(y_test,y_pred)}")#measure R-squared value
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print(f"Weights:{ann_model.get_weights()}")
    print(f"Intercept: {mlr_model.intercept_}")
    print(f"Coefficients: {list(zip(mlr_model.coef_))}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

3/3 [==============================] - 0s 4ms/step
MAE: 0.3027212316654919
R2: 0.1018365788484219
RMSE: 0.3770531253363837
Weights:[array([[-0.26072642],
       [ 0.7375939 ],
       [-2.3246753 ],
       [ 1.0522021 ],
       [-1.066334  ],
       [-0.8947814 ],
       [ 0.1784551 ],
       [-0.22856286],
       [-2.359577  ],
       [-0.05769231],
       [ 0.04867163],
       [ 0.14839718],
       [-0.16484109],
       [-0.04613088],
       [-0.14418076],
       [ 0.16685432]], dtype=float32), array([-0.9466471], dtype=float32), array([[0.8673335]], dtype=float32), array([0.7078924], dtype=float32)]
Intercept: 0.7947303437178834
Coefficients: [(0.05313095918327541,), (-0.5476592649180987,), (-0.6118874643581756,), (0.5202472072796256,), (-0.22289618143232243,), (-0.1670847080613103,), (-0.057585813711760366,), (0.1319028051044666,), (-0.5155230783302909,), (-0.03155879655941255,), (0.04129673557420371,), (0.050837414804058415,), (0.1752598375244271,), (-0.013879427563714727,), (-0.06